In [1]:
from bitstring import Bits
import numpy as np




###################################
####################################
rows = 3         #3
collumns = 3     #4
##########################################
#########################################



#number of bits in each value
num_of_bits = 32
#
scale =12


print(f"You are computing the DFT of N = {rows * collumns}.")



You are computing the DFT of N = 9.


# Auxiliar functions

In [2]:

# Converts an integer value to a twos complement value
def integer_to_binary_2complements(float_value , scale=12, num_of_bits = 32):
    return str(Bits(int=round(float_value*(2**scale)), length=num_of_bits)).replace("0x","")


## Computes the twiddle factors MATRIX

In [3]:


# Computes the DFT factors of a N size vector
def dft_factors(N):
    """ Obtem os valores das constantes"""
    X = np.zeros(N, dtype=complex)
    twiddle_n   =  []
    for k in range(N):
        for n in range(N):
            twiddle_n.append(np.exp(-2j * np.pi * k * n / N))
        #    print(np.exp(-2j * np.pi * k * n / N))
        #print('\n')
    print("\n\n\n")
    #print(twiddle_n)
    return twiddle_n

# reshapes data
def format_data_properly(X, N):
    X = np.array(X).reshape(N, N)
    return X

def create_twiddle_std_logic_values(row_length,collumn_length):	
    N = row_length*collumn_length
    twiddle_code = "CONSTANT twiddle_matrix : MATRIX :=\n ("
    for i in np.arange(0,collumn_length):
        twiddle_code += "("
        for k in np.arange(0,row_length):
            num = np.exp(-2j*i*k*np.pi/N)
            #print(num)
            twiddle_code += "(x\"" +integer_to_binary_2complements(num.real, scale, num_of_bits)+ "\",x\"" + integer_to_binary_2complements(num.imag,scale,num_of_bits).replace("(", "") + "\")"
            #print(float_bin(round(num.real,4),8))
            #twiddle_code += '('+float_bin(xx , 8)
            if k == row_length-1:
                twiddle_code += "),\n"
            else:
                twiddle_code += ","              
    twiddle_code += ")"
    s = list(twiddle_code)
    s[-3] = ''
    twiddle_code = "".join(s)
    twiddle_code += ";"
    print(twiddle_code)
    return twiddle_code



create_twiddle_std_logic_values(collumns,rows)




CONSTANT twiddle_matrix : MATRIX :=
 (((x"00001000",x"00000000"),(x"00001000",x"00000000"),(x"00001000",x"00000000")),
((x"00001000",x"00000000"),(x"00000c42",x"fffff5b7"),(x"000002c7",x"fffff03e")),
((x"00001000",x"00000000"),(x"000002c7",x"fffff03e"),(x"fffff0f7",x"fffffa87"))
);


'CONSTANT twiddle_matrix : MATRIX :=\n (((x"00001000",x"00000000"),(x"00001000",x"00000000"),(x"00001000",x"00000000")),\n((x"00001000",x"00000000"),(x"00000c42",x"fffff5b7"),(x"000002c7",x"fffff03e")),\n((x"00001000",x"00000000"),(x"000002c7",x"fffff03e"),(x"fffff0f7",x"fffffa87"))\n);'

# Determins the row and collumn DFT factors.

In [4]:

# created the tring with the twiddle factors in signed type twos complement
def row_dft_collumn_factors(X, row_or_collumn, type_of_vector): 
    twiddle_code = f"\n\nCONSTANT {row_or_collumn}_dft_matrix_values : {type_of_vector.upper()}_MATRIX :=\n (("
    for row in X:
        for num in row:
            twiddle_code += ",(x\"" +integer_to_binary_2complements(num.real,scale, num_of_bits)+ "\",x\"" + integer_to_binary_2complements(num.imag,scale, num_of_bits).replace("(", "") + "\")"
        twiddle_code += "),\n("
    twiddle_code += ");"
    print(twiddle_code.replace(",\n();",");").replace("(,(","(("))
    return twiddle_code.replace(",\n();",");").replace("(,(","((")


# Final method to compute the tf of the rows and collumns
def row_or_collumn(N, row_or_collumn = "row", type_of_vector= "collumn"):

    X = dft_factors(N)
    X = format_data_properly(X,N)
    return row_dft_collumn_factors(X,row_or_collumn, type_of_vector)


row_or_collumn(rows, "rows", "collumn")

row_or_collumn(collumns, "collumn" , "row")







CONSTANT rows_dft_matrix_values : COLLUMN_MATRIX :=
 (((x"00001000",x"00000000"),(x"00001000",x"00000000"),(x"00001000",x"00000000")),
((x"00001000",x"00000000"),(x"fffff800",x"fffff225"),(x"fffff800",x"00000ddb")),
((x"00001000",x"00000000"),(x"fffff800",x"00000ddb"),(x"fffff800",x"fffff225")));






CONSTANT collumn_dft_matrix_values : ROW_MATRIX :=
 (((x"00001000",x"00000000"),(x"00001000",x"00000000"),(x"00001000",x"00000000")),
((x"00001000",x"00000000"),(x"fffff800",x"fffff225"),(x"fffff800",x"00000ddb")),
((x"00001000",x"00000000"),(x"fffff800",x"00000ddb"),(x"fffff800",x"fffff225")));


'\n\nCONSTANT collumn_dft_matrix_values : ROW_MATRIX :=\n (((x"00001000",x"00000000"),(x"00001000",x"00000000"),(x"00001000",x"00000000")),\n((x"00001000",x"00000000"),(x"fffff800",x"fffff225"),(x"fffff800",x"00000ddb")),\n((x"00001000",x"00000000"),(x"fffff800",x"00000ddb"),(x"fffff800",x"fffff225")));'

# Save to text

In [5]:



with open('factors.txt', 'w') as f:
    f.write("\n")
    f.write(create_twiddle_std_logic_values(collumns,rows))
    f.write(row_or_collumn(rows, "collumn" , "row"))
    f.write("\n")
    f.write(row_or_collumn(collumns, "row" , "collumn"))
    f.write("\n")


CONSTANT twiddle_matrix : MATRIX :=
 (((x"00001000",x"00000000"),(x"00001000",x"00000000"),(x"00001000",x"00000000")),
((x"00001000",x"00000000"),(x"00000c42",x"fffff5b7"),(x"000002c7",x"fffff03e")),
((x"00001000",x"00000000"),(x"000002c7",x"fffff03e"),(x"fffff0f7",x"fffffa87"))
);






CONSTANT collumn_dft_matrix_values : ROW_MATRIX :=
 (((x"00001000",x"00000000"),(x"00001000",x"00000000"),(x"00001000",x"00000000")),
((x"00001000",x"00000000"),(x"fffff800",x"fffff225"),(x"fffff800",x"00000ddb")),
((x"00001000",x"00000000"),(x"fffff800",x"00000ddb"),(x"fffff800",x"fffff225")));






CONSTANT row_dft_matrix_values : COLLUMN_MATRIX :=
 (((x"00001000",x"00000000"),(x"00001000",x"00000000"),(x"00001000",x"00000000")),
((x"00001000",x"00000000"),(x"fffff800",x"fffff225"),(x"fffff800",x"00000ddb")),
((x"00001000",x"00000000"),(x"fffff800",x"00000ddb"),(x"fffff800",x"fffff225")));


In [6]:
def tex_initial(rows, collumns):
    return f"""library IEEE;
USE IEEE.STD_LOGIC_1164.ALL;
USE ieee.numeric_std.ALL;      -- going to use to convert std vector to unsigned


-- Package Declaration Section-0.02112+0.j
package Packages_Util is
    
    
    CONSTANT  rows :     INTEGER :=  {rows};
    CONSTANT  collumns : INTEGER :=  {collumns};
    
    
    ----------------------------------------------------------
    ------              Clocks                         -------
    ----------------------------------------------------------
   
    
    --clk generation.For 100 MHz clock this generates 1 Hz clock.
    CONSTANT acquisition_clock_counter : INTEGER := 100000000;
    CONSTANT output_clock_counter      : INTEGER := 100000000;
 
 
    ----------------------------------------------------------
    ------             Data Types                      -------
    ----------------------------------------------------------
   
 
	TYPE Complex_Type IS
		RECORD
			r: signed(31 DOWNTO 0);
			i: signed(31 DOWNTO 0);
		END RECORD;
		
    TYPE VECTOR_COLLUMN IS ARRAY (0 to collumns-1) OF Complex_Type; 
    TYPE VECTOR_ROW     IS ARRAY (0 to rows-1)     OF Complex_Type; 
    
    TYPE MATRIX         IS ARRAY (0 TO rows-1) OF VECTOR_COLLUMN;
    TYPE MATRIX_transpose  IS ARRAY (0 TO collumns-1) OF VECTOR_ROW;

    TYPE ROW_MATRIX     IS ARRAY (0 TO rows-1) OF VECTOR_ROW;
    TYPE COLLUMN_MATRIX IS ARRAY (0 TO collumns-1) OF VECTOR_COLLUMN;

    ----------------------------------------------------------
    ------           Pre Computed Matrix               -------
    ----------------------------------------------------------
    -- os valores estão calculados em 2s complement " """

    


text_final =  """
    -------------------------------------------------------
    ------        Deslaração de Funções             -------
    -------------------------------------------------------
      

   -- Soma de numeros complexos
   ------------------------------------------------
	FUNCTION ComplexSum (ValueA, ValueB: Complex_Type) RETURN Complex_Type;
	------------------------------------------------

    -- Multiplciação de numeros complexos
	------------------------------------------------
	FUNCTION ComplexMULT (ValueA, ValueB: Complex_Type) RETURN Complex_Type;
	------------------------------------------------
	

	
      end package Packages_Util;
       
      -- Package Body Section
      package body Packages_Util is
       
	
	------------------------------------------------
	--Calcula a soma de dois numeros complexos
	FUNCTION ComplexSum (ValueA, ValueB: Complex_Type) RETURN Complex_Type IS
		
		VARIABLE Result : Complex_Type;
        VARIABLE Natural_result : signed(31 downto 0);
        VARIABLE Complex_result : signed(31 downto 0);
    
	BEGIN
	
		Natural_result := ValueA.r + ValueB.r;
		Complex_result := ValueA.i + ValueB.i;
		
        Result.r := Natural_result(31 downto 0);
        Result.i := Complex_result(31 downto 0);
		RETURN Result;
		
	END ComplexSum;
	

	------------------------------------------------
	-- Calcula o produto entre dois numeros complexos
    FUNCTION ComplexMult(ValueA, ValueB: Complex_Type) RETURN Complex_Type IS
        
        VARIABLE Result: Complex_Type;
        VARIABLE Natural_result_first : signed(63 downto 0);
        VARIABLE Complex_result_first : signed(63 downto 0);
        VARIABLE Natural_result_second : signed(63 downto 0);
        VARIABLE Complex_result_second : signed(63 downto 0);   
        VARIABLE Natural_result : signed(63 downto 0);
        VARIABLE Complex_result : signed(63 downto 0); 
    BEGIN
    
        Natural_result_first := signed(ValueA.r) *signed(ValueB.r)/4096 ;
        Complex_result_first := signed(ValueA.r) *signed(ValueB.i)/4096 ;
   
        Natural_result_second :=  signed(ValueA.i)*signed(ValueB.i)/4096;
        Complex_result_second :=  signed(ValueA.i)*signed(ValueB.r)/4096;
     
     
        Natural_result := Natural_result_first- Natural_result_second;
        Complex_result := Complex_result_first+ Complex_result_second;
     
        Result.r := Natural_result(31 downto 0);
        Result.i := Complex_result(31 downto 0);
    
        RETURN Result;
    END ComplexMult;
	
 
    
END package body Packages_Util;"""



with open('factors.txt', 'w') as f:
    f.write(tex_initial(rows, collumns))
    f.write("\n")
    f.write(create_twiddle_std_logic_values(collumns,rows))
    f.write(row_or_collumn(rows, "collumn" , "row"))
    f.write("\n")
    f.write(row_or_collumn(collumns, "row" , "collumn"))
    f.write(text_final)

CONSTANT twiddle_matrix : MATRIX :=
 (((x"00001000",x"00000000"),(x"00001000",x"00000000"),(x"00001000",x"00000000")),
((x"00001000",x"00000000"),(x"00000c42",x"fffff5b7"),(x"000002c7",x"fffff03e")),
((x"00001000",x"00000000"),(x"000002c7",x"fffff03e"),(x"fffff0f7",x"fffffa87"))
);






CONSTANT collumn_dft_matrix_values : ROW_MATRIX :=
 (((x"00001000",x"00000000"),(x"00001000",x"00000000"),(x"00001000",x"00000000")),
((x"00001000",x"00000000"),(x"fffff800",x"fffff225"),(x"fffff800",x"00000ddb")),
((x"00001000",x"00000000"),(x"fffff800",x"00000ddb"),(x"fffff800",x"fffff225")));






CONSTANT row_dft_matrix_values : COLLUMN_MATRIX :=
 (((x"00001000",x"00000000"),(x"00001000",x"00000000"),(x"00001000",x"00000000")),
((x"00001000",x"00000000"),(x"fffff800",x"fffff225"),(x"fffff800",x"00000ddb")),
((x"00001000",x"00000000"),(x"fffff800",x"00000ddb"),(x"fffff800",x"fffff225")));


In [7]:
      
#         file Fin : text open READ_MODE  is "datain.txt";
    
#         variable current_read_line : line;
#         variable current_read_field : std_logic_vector(12 downto 0);
        
      
            
            
#             while (not endfile(Fin)) loop
            
#                 readline(Fin, current_read_line);
#                 read(current_read_line, current_read_field);
                
#                 NEW_VALUE <=  current_read_field;
#                 wait for clock_period;

#             end loop;



import numpy as np

np.fft.fft(np.array([0.735, 0.745, 0.728, 0.73912]))




array([ 2.94712+0.j     ,  0.007  -0.00588j, -0.02112+0.j     ,
        0.007  +0.00588j])